In [2]:
!pip install tensorflow tensorflow-gpu pandas matplotlib sklearn opencv-python-headless

# Importing required packages
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import cv2
from matplotlib import pyplot as plt
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached protobuf-3.19.6-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "<string>", line 36, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-12t_vpbi/tensorflow-gpu_1bcb501ca4144c959e23d2ef2c786263/setup.py", line 37, in <module>
          raise Exception(TF_REMOVAL_WARNING)
      Exception:
      
      The "tensorflow-gpu" package has been removed!
      
      Please install "tensorflow" instead.
      
      Other than the name, the two packages have been identical
      since TensorFlow 2.1, or roughly since Sep 2019. For more
      information, see: pypi.org/project/tensorflow-gpu
      
      
      [end of outpu

In [3]:
!pip install gradio jinja2

  Using cached gradio-3.24.1-py3-none-any.whl (15.7 MB)
  Using cached aiofiles-23.1.0-py3-none-any.whl (14 kB)
  Using cached httpx-0.23.3-py3-none-any.whl (71 kB)
  Using cached python_multipart-0.0.6-py3-none-any.whl (45 kB)
  Using cached gradio_client-0.0.7-py3-none-any.whl (14 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached linkify_it_py-1.0.3-py3-none-any.whl (19 kB)
  Using cached httpcore-0.16.3-py3-none-any.whl (69 kB)


In [4]:
import gradio as gr

In [6]:
# Reading the dataset into a pandas dataframe
df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv')
X = df['comment_text']
y = df[df.columns[2:]].values

In [7]:
# Defining the maximum number of features
MAX_FEATURES = 200000

In [8]:
# Creating a TextVectorization layer for encoding and adapting the training dataset
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=max([len(x.split()) for x in X.values]),
                               output_mode='int')
vectorizer.adapt(X.values)

In [9]:
# Creating a TensorFlow dataset from the vectorized text and labels
dataset = tf.data.Dataset.from_tensor_slices((vectorizer(X.values), y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps with bottlenecks

In [10]:
# Splitting dataset into train, validation, and test sets
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [11]:
# Defining a Bidirectional LSTM model
model = Sequential([
    Embedding(MAX_FEATURES+1, 32),
    Bidirectional(LSTM(32, activation='tanh')),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(6, activation='sigmoid')
])

In [12]:
# Compiling the model using Binary Cross Entropy loss function and Adam optimizer
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [13]:
# Define model file path
model_filepath = 'toxicity.h5'

In [14]:
# Check if a pre-trained model exists and load it
if os.path.exists(model_filepath):
    print("Loading pre-trained model...")
    model = tf.keras.models.load_model(model_filepath)
else:
    print("Training the model...")
    history = model.fit(train, epochs=1, validation_data=val)
    model.save(model_filepath)

Training the model...
70/70 [==============================] - 63s 821ms/step - loss: 0.2530 - val_loss: 0.1503


In [15]:
# Evaluation metrics
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [16]:
# Calculate Precision, Recall, and Categorical Accuracy scores for the model
for batch in test.as_numpy_iterator(): 
    X_true, y_true = batch
    yhat = model.predict(X_true)
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 140ms/step


In [17]:
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision:0.0, Recall:0.0, Accuracy:0.30000001192092896


In [21]:
# Defining a 'score_comment' function to predict toxicity labels for an input comment,
# vectorize the comment, get predictions, and format the results as a string.
from io import BytesIO
import base64

def score_comment(comment, category=None, custom=None):
    vectorized_comment = vectorizer([comment])
    
    if category and custom:
        # Add custom phrase or word to the input data
        X_custom = np.zeros((1, max_sequence_length), dtype=np.int64)
        custom_text = '{} {}'.format(category, custom)
        vectorized_custom = vectorizer([custom_text])
        X_custom[:, :len(vectorized_custom[0])] = vectorized_custom
        X = np.vstack([vectorized_comment, X_custom])
    else:
        X = vectorized_comment
    
    results = model.predict(X)[-1]

    # Create a bar chart of category probabilities
    fig, ax = plt.subplots()
    y_pos = np.arange(len(df.columns[2:]))
    ax.barh(y_pos, results, align='center')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(df.columns[2:])
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_xlabel('Probability')
    ax.set_title('Category Probabilities')

    # Save the plot to a buffer
    buf = BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)

    # Convert the buffer to a base64 encoded string
    encoded_img = base64.b64encode(buf.getvalue()).decode('utf-8')
    plt.close(fig)

    # Convert base64 image to numpy array
    nparr = np.frombuffer(base64.b64decode(encoded_img), np.uint8)
    img_np = cv2.imdecode(nparr, cv2.IMREAD_UNCHANGED)

    return img_np

In [22]:
# Creating an interface for the model using Gradio to take user input,
# display predictions and probability distributions for different categories,
# and provide a visual output.
interface = gr.Interface(fn=score_comment,
                         inputs=[gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                                 gr.inputs.CheckboxGroup(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], label='Categories'),
                                 gr.inputs.Textbox(label='Custom phrase or word', optional=True)],
                         outputs=gr.outputs.Image(type='pil'))

/opt/conda/lib/python3.7/site-packages/gradio/inputs.py:28: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/opt/conda/lib/python3.7/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/opt/conda/lib/python3.7/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/opt/conda/lib/python3.7/site-packages/gradio/inputs.py:150: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/opt

In [23]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://712155ba6c07af5567.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


ReadTimeout: HTTPSConnectionPool(host='712155ba6c07af5567.gradio.live', port=443): Read timed out. (read timeout=3)